In [1]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
deploy_type = "local"

config_dict = json.load(Path("../experiments/config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

base_path


PosixPath('/home/rzlin/ys64ofuj/Data/StressGait/Data')

In [3]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True, specify_speed=False)
index = dataset.create_index()

In [4]:
# For all other participants
vp_list = ["VP_11", "VP12", "VP13"]
mask_other = ~index["participant"].isin(vp_list)
index.loc[mask_other, "speed"] = index.loc[mask_other, "bout"].apply(lambda x: "slow" if x == "1" else "fast")

# For participants in the list
mask_vp = index["participant"].isin(vp_list)
index.loc[mask_vp, "speed"] = index.loc[mask_vp, "bout"].apply(lambda x: "fast" if x == "1" else "slow")

In [5]:
index

,participant,condition,bout,speed
0,VP_01,omc,1,slow
1,VP_01,omc,2,fast
2,VP_02,control,1,slow
3,VP_02,control,2,fast
4,VP_05,control,1,slow
...,...,...,...,...
77,VP_45,omc,2,fast
78,VP_46,control,1,slow
79,VP_46,control,2,fast
80,VP_47,omc,1,slow


In [6]:
subset = dataset[0]
subset.kinematics

hip_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                
VP_01       omc       1    slow  0         0                       17.179218   
                                           1                       16.507507   
                                           2                       15.723399   
                                           3                       14.842070   
                                           4                       13.871923   
...                                                                      ...   
                                           96                      15.332522   
                                           97                      14.753352   
                                           98                      14.275163   
                                           99                      13.884479   
                                           100                     13.543158   

                                                                 knee_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                 
VP_01       omc       1    slow  0         0                        12.116574   
                                           1                        11.065104   
                                           2                        10.305527   
                                           3                         9.732018   
                                           4                         9.200402   
...                                                                       ...   
                                           96                       10.740914   
                                           97                        7.412251   
                                           98                        4.806047   
                                           99                        3.150390   
                                           100                       2.030940   

                                                                 ankle_plantarflexion  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                               -11.059877   
                                           1                                -9.919752   
                                           2                                -8.658710   
                                           3                                -7.326190   
                                           4                                -5.992229   
...                                                                               ...   
                                           96                               -2.231804   
                                           97                               -1.460848   
                                           98                               -0.583145   
                                           99                                0.188064   
                                           100                               0.899200   

                                                                 elbow_flexion_angles  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                                71.313000   
                                           1                                71.139635   
                                           2                                70.885186   
                                           3                                70.569000   
                                           4                                70.212616   
...                                                                               ...   
                                           96                                9.971010   
           

In [10]:
index["speed"] = ["fast" if index.loc[i, 'bout'] == '1' else "slow" for i in range(len(index)) if index.loc[i, "participant"] in ["VP11"] ]


SyntaxError: invalid syntax (1914414107.py, line 1)

In [3]:
dataset[0].load_force_plate_data()

NameError: name 'dataset' is not defined

In [5]:
dataset[0].load_keypoint_trajectories()


,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
time,,,,,,,,,,,,,,,,,,,,,
0.00,250.261279,669.129684,243.341368,677.954668,262.537737,670.098222,237.326185,681.952762,273.366112,664.638598,...,277.606317,424.553234,209.905593,257.543759,247.905952,256.932118,204.818913,106.459732,214.978603,122.693037
0.02,250.483644,667.943734,243.432801,676.289324,262.695375,668.771226,237.691338,680.607385,273.679450,663.569858,...,278.128968,426.039994,209.959943,257.210277,247.832269,255.814951,204.558580,106.266183,215.907242,122.529621
0.04,251.656972,667.215227,244.443433,675.827329,263.962387,668.774995,238.275877,679.069058,274.723592,663.147516,...,278.338266,426.837565,209.909766,256.809683,248.034691,255.262220,204.209230,106.424710,216.284980,122.466593
0.06,253.445246,666.843777,245.988398,676.276591,265.948232,669.798178,238.827390,677.336805,276.205690,663.249749,...,278.364126,427.333333,209.745400,256.270255,248.470098,255.177677,203.871027,106.709353,216.211540,122.667706
0.08,254.570617,666.452979,246.871784,676.477262,267.188925,670.545035,238.899603,675.760092,277.079949,663.322568,...,278.448076,427.916458,209.558363,255.730466,248.743285,255.116863,203.695977,106.744754,216.140172,123.089461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250.28,331.949575,675.169230,324.942422,683.004462,330.504879,680.022844,313.090211,680.101241,318.585388,671.567337,...,278.045744,425.400289,356.954012,266.055230,266.883098,259.749887,335.744810,115.602991,247.943346,115.583391
250.30,333.266545,671.494279,327.620016,677.548283,334.461791,675.276989,314.616229,676.008539,323.298158,668.816511,...,281.135239,425.952344,353.019519,263.360218,263.274435,260.850365,349.929165,111.317458,241.678528,117.519316
250.32,334.482757,668.139670,329.035852,673.676744,338.946233,670.261123,315.560872,672.018338,326.787505,664.682952,...,282.310895,426.153188,348.705894,260.678397,260.163013,261.143654,362.668722,109.784805,235.329998,119.792411


import numpy as np

In [ ]:
#how to use the multilevel kinematics dataframe

kinematics_df.xs(("VP_01", "omc", "1"), drop_level=False)
#df.xs(("one", "bar"), level=("second", "first"), axis=1)